In [4]:
with open("/home/davide/Downloads/KEGGCCMEE.html") as f:
    lines = f.readlines()

#use beautifulsoup to parse the html
from bs4 import BeautifulSoup
soup = BeautifulSoup(open("/home/davide/Downloads/KEGGCCMEE.html"), 'html.parser')
#find all a tags and all b tags
text=soup.get_text()
text=text[:text.find("Organismal Systems")]
lines = text.split("\n")
diz = {}
for i,line in enumerate(lines):
    if line.startswith("0"):
        if "(" in line:
            current_key = line[:line.index("(")].strip()
        else:
            current_key = line
            #add to key lines[i+j] until lines[i+j] starts has (
            j = 1
            while not " (" in lines[i+j]:
                current_key += lines[i+j]
                j += 1
        diz[current_key] = []

    elif line.startswith("WP_"):
        proteins = line.split(", ") if ", " in line else [line]
        diz[current_key].extend(proteins)
#now make a dataframe that for every value of the dictionary has a row with the key as index and the keys of the dictionary in which appears as columns
import pandas as pd

# Get a list of all unique protein IDs
all_proteins = set(protein for proteins in diz.values() for protein in proteins)

# Initialize an empty list to hold the dataframes
dfs = []

# Iterate over each protein in the set of all proteins
for protein in all_proteins:
    # Create a dictionary where the keys are the pathways and the values are 1 if the protein is present in the pathway and 0 otherwise
    data = {pathway: 1 if protein in proteins else 0 for pathway, proteins in diz.items()}
    # Convert the dictionary into a DataFrame and append it to the list
    df = pd.DataFrame(data, index=[protein])
    dfs.append(df)

# Concatenate all the dataframes into a single dataframe
result = pd.concat(dfs)

In [5]:
result

,01100 Metabolic pathways,01110 Biosynthesis of secondary metabolites,01120 Microbial metabolism in diverse environments,01200 Carbon metabolism,01210 2-Oxocarboxylic acid metabolism,01212 Fatty acid metabolism,01230 Biosynthesis of amino acids,01232 Nucleotide metabolism,01250 Biosynthesis of nucleotide sugars,01240 Biosynthesis of cofactors,...,04113 Meiosis - yeast,04214 Apoptosis - fly,04216 Ferroptosis,04217 Necroptosis,02024 Quorum sensing,05111 Biofilm formation - Vibrio cholerae,02025 Biofilm formation - Pseudomonas aeruginosa,02026 Biofilm formation - Escherichia coli,02030 Bacterial chemotaxis,02040 Flagellar assembly
,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_250122003.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_250121290.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_250126236.1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_250123796.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_250121169.1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_250126067.1,1,1,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
WP_250126377.1,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_250123384.1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
result.columns

Index(['01100 Metabolic pathways',
       '01110 Biosynthesis of secondary metabolites',
       '01120 Microbial metabolism in diverse environments',
       '01200 Carbon metabolism', '01210 2-Oxocarboxylic acid metabolism',
       '01212 Fatty acid metabolism', '01230 Biosynthesis of amino acids',
       '01232 Nucleotide metabolism',
       '01250 Biosynthesis of nucleotide sugars',
       '01240 Biosynthesis of cofactors',
       ...
       '04113 Meiosis - yeast', '04214 Apoptosis - fly', '04216 Ferroptosis',
       '04217 Necroptosis', '02024 Quorum sensing',
       '05111 Biofilm formation - Vibrio cholerae',
       '02025 Biofilm formation - Pseudomonas aeruginosa',
       '02026 Biofilm formation - Escherichia coli',
       '02030 Bacterial chemotaxis', '02040 Flagellar assembly'],
      dtype='object', length=172)

In [7]:
fimo_best="CCMEE29best_site.narrowPeak"
genbank_dir = "annotati_Refseq"
import os
import re
from Bio import SeqIO
from collections import Counter
from scipy.stats import hypergeom
import pandas as pd
diz={

    'LAU37' : 'Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.gbff' ,

}
with open(fimo_best,"r") as f:
    content=f.readlines()
names = [line.split()[0] for line in content]
kos = []
kos_background = []
ko_to_genes = {}
i,j=0,0
for file2 in os.listdir(genbank_dir):
    genoma = SeqIO.parse(os.path.join(genbank_dir,file2), "genbank")
    for seq in genoma:
        for record in seq.features:
            if record.type != "CDS":
                continue
            gene=record.qualifiers.get("gene", ["None"])[0]
            pid=record.qualifiers.get("protein_id", ["None"])[0]
            if record.qualifiers["locus_tag"][0] in names:
                i+=1
                #see row of result with the protein id as index
                if pid in result.index:
                    #make a list of columns of the row with value 1
                    ko_terms = result.columns[result.loc[pid] == 1].tolist()
                    for ko_term in ko_terms:
                        kos.append(ko_term)
                        if ko_term in ko_to_genes:
                            ko_to_genes[ko_term].append(gene)
                        else:
                            ko_to_genes[ko_term] = [gene]
            else:
                if pid in result.index:
                    j+=1
                    kos_background.extend(result.columns[result.loc[pid] == 1].tolist())
kos = Counter(kos)
kos_background = Counter(kos_background)
M = i+j
n = i
p_values = {}
enrichments = {}
for ko in kos:
    k = kos[ko]
    N = kos_background[ko]
    p_values[ko] = hypergeom.sf(k-1, M, N, n)
    enrichments[ko] = (k/n) / (N/M)
    enrichments[ko] = "positivo" if enrichments[ko] > 1 else "negativo"
df=pd.DataFrame.from_dict(p_values,orient="index",columns=["p_value"])
df["enrichment"] = pd.Series(enrichments)
for ko in kos:
    df.at[ko, "genes with motif"] = ", ".join(ko_to_genes[ko])
df.sort_values("p_value", inplace=True, ascending=True)






In [23]:
df

,p_value,enrichment,genes with motif
03440 Homologous recombination,0.109839,positivo,"None, recA, None, None"
"00280 Valine, leucine and isoleucine degradation",0.169068,positivo,"lpdA, None"
03430 Mismatch repair,0.255539,positivo,"None, xseA, None"
03030 DNA replication,0.317363,positivo,"None, None"
00730 Thiamine metabolism,0.366289,positivo,"None, thiD"
00640 Propanoate metabolism,0.413816,positivo,"lpdA, None"
00430 Taurine and hypotaurine metabolism,0.417648,positivo,ggt
00340 Histidine metabolism,0.459517,positivo,"None, hisD"
00310 Lysine degradation,0.491413,positivo,lpdA
00380 Tryptophan metabolism,0.612220,positivo,lpdA


In [24]:
print(i,j)

171 1183


In [8]:
kos

Counter({'01100 Metabolic pathways': 14,
         '01110 Biosynthesis of secondary metabolites': 5,
         '03440 Homologous recombination': 4,
         '01240 Biosynthesis of cofactors': 4,
         '03430 Mismatch repair': 3,
         '03030 DNA replication': 2,
         '00195 Photosynthesis': 2,
         '01210 2-Oxocarboxylic acid metabolism': 2,
         '01230 Biosynthesis of amino acids': 2,
         '02020 Two-component system': 2,
         '01120 Microbial metabolism in diverse environments': 2,
         '01200 Carbon metabolism': 2,
         '00630 Glyoxylate and dicarboxylate metabolism': 2,
         '00640 Propanoate metabolism': 2,
         '00280 Valine, leucine and isoleucine degradation': 2,
         '00730 Thiamine metabolism': 2,
         '00340 Histidine metabolism': 2,
         '03010 Ribosome': 2,
         '05111 Biofilm formation - Vibrio cholerae': 1,
         '02026 Biofilm formation - Escherichia coli': 1,
         '01250 Biosynthesis of nucleotide sugars': 

In [9]:
kos_background

Counter({'01100 Metabolic pathways': 760,
         '01110 Biosynthesis of secondary metabolites': 346,
         '01240 Biosynthesis of cofactors': 158,
         '01120 Microbial metabolism in diverse environments': 157,
         '02010 ABC transporters': 117,
         '01230 Biosynthesis of amino acids': 111,
         '01200 Carbon metabolism': 85,
         '02020 Two-component system': 67,
         '00195 Photosynthesis': 67,
         '01250 Biosynthesis of nucleotide sugars': 65,
         '03010 Ribosome': 54,
         '00860 Porphyrin metabolism': 51,
         '00520 Amino sugar and nucleotide sugar metabolism': 43,
         '00190 Oxidative phosphorylation': 42,
         '00230 Purine metabolism': 40,
         '00541 O-Antigen nucleotide sugar biosynthesis': 39,
         '02024 Quorum sensing': 35,
         '00500 Starch and sucrose metabolism': 32,
         '00270 Cysteine and methionine metabolism': 30,
         '00630 Glyoxylate and dicarboxylate metabolism': 30,
         '00400